# Desafio 1 - SQL

Dado um banco de dados de votos ganhos por diferentes candidatos em uma eleição, encontre o número de votos ganhos por candidatas do sexo feminino cuja idade é inferior a 50 anos.

São Duas Tabelas Cantidates e Results

**Schemas**:

`Candidates`

| Name | Type | Description|
| ---- | ---- | -----------|
| id | INTEGER | It is the primary key.|
| gender | STRING | The gender of the candidate.|
| age | INTEGER | Age of the candidate.|
| party | STRING | The party to which the candidate belongs.|

`Results`

| Name | Type | Description|
| ---- | ---- | -----------|
| constituency_id | INTEGER | It is the constituency to which the candidate is contesting from.|
| candidate_id | INTEGER | It is the primary key.|
| votes | INTEGER | The number of votes won by the candidate.|

**Dados da Tabela**

`Candidates`

| id | gender | age | party |
|----|--------|-----|-------|
| 1 | M | 55 |Democratic |
| 2 | M | 51 |Democratic |
| 3 | F | 49 |Democratic |
| 4 | M | 60 |Republic |
| 5 | F | 61 |Republic |
| 6 | F | 48 |Republic |

`Results`
| constituency_id | candidate_id | votes |
|-----------------|--------------|-------|
| 1 | 1 | 847529 | 
| 1 | 4 | 283409 | 
| 2 | 2 | 293841 | 
| 2 | 5 | 394385 | 
| 3 | 3 | 429084 | 
| 3 | 6 | 303890 | 

## Inicialização do banco de dados

In [1]:
import sqlite3

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

db_name = "election.db"
conn = create_connection(db_name)
cursor = conn.cursor()

## Criação das tabelas

In [2]:
table_names = ["Candidates", "Results"]

for table_name in table_names:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    except sqlite3.OperationalError as e:
        print(f"Error dropping table '{table_name}': {e}")

In [3]:
cursor.execute('''
CREATE TABLE Candidates (
    id INTEGER PRIMARY KEY,
    gender TEXT,
    age INTEGER,
    party TEXT
)
''')

cursor.execute('''
CREATE TABLE Results (
    constituency_id INTEGER,
    candidate_id INTEGER,
    votes INTEGER,
    PRIMARY KEY (constituency_id, candidate_id)
)
''')

conn.commit()

In [4]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info('{table_name}')")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Candidates
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'gender', 'TEXT', 0, None, 0)
(2, 'age', 'INTEGER', 0, None, 0)
(3, 'party', 'TEXT', 0, None, 0)


Table: Results
(0, 'constituency_id', 'INTEGER', 0, None, 1)
(1, 'candidate_id', 'INTEGER', 0, None, 2)
(2, 'votes', 'INTEGER', 0, None, 0)




## Inserção de dados

In [5]:
candidates_data = [
    (1, 'M', 55, 'Democratic'),
    (2, 'M', 51, 'Democratic'),
    (3, 'F', 49, 'Democratic'),
    (4, 'M', 60, 'Republic'),
    (5, 'F', 61, 'Republic'),
    (6, 'F', 48, 'Republic')
]

cursor.executemany('''
INSERT INTO Candidates (id, gender, age, party)
VALUES (?, ?, ?, ?)
''', candidates_data)

results_data = [
    (1, 1, 847529),
    (4, 2, 283409),
    (3, 3, 293841),
    (2, 5, 394385),
    (3, 4, 429084),
    (6, 6, 303890)
]

cursor.executemany('''
INSERT INTO Results (constituency_id, candidate_id, votes)
VALUES (?, ?, ?)
''', results_data)

conn.commit()


In [6]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"SELECT * FROM {table_name}")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Candidates
(1, 'M', 55, 'Democratic')
(2, 'M', 51, 'Democratic')
(3, 'F', 49, 'Democratic')
(4, 'M', 60, 'Republic')
(5, 'F', 61, 'Republic')
(6, 'F', 48, 'Republic')


Table: Results
(1, 1, 847529)
(4, 2, 283409)
(3, 3, 293841)
(2, 5, 394385)
(3, 4, 429084)
(6, 6, 303890)




## Consulta

In [7]:
query = '''
SELECT SUM(R.votes) AS total_votes
FROM Candidates C
JOIN Results R ON C.id = R.candidate_id
WHERE C.gender = 'F' AND C.age < 50
'''

cursor.execute(query)
result = cursor.fetchone()

print(f"Total de votos ganhos por candidatas do sexo feminino com menos de 50 anos: {result[0]}")


Total de votos ganhos por candidatas do sexo feminino com menos de 50 anos: 597731


In [8]:
conn.close()